## **Image-Based Receipt Extraction with Orq**

---



Handling unstructured data at scale is a common challenge, particularly when dealing with formats like .jpg and .png. Orq provides a robust solution for transforming these images into actionable data. This guide walks through the process of encoding images, sending them to Orq for processing, and extracting structured outputs efficiently. Whether it’s a handful of receipts or a large batch, this workflow ensures accuracy and scalability.

To make things even easier, we’ve created this [Google Colab file](https://colab.research.google.com/drive/1ZJI7hbKanDKo14R1sh1D9HwPF0RENA42#scrollTo=yGQknhoWoM_B) that you can copy and run straight away after replacing the API key—the deployment is already live and ready in the deployment section. Below, we’ll run through the code step by step for further explanation.

Ready to unlock Orq's magic?[ Sign up](https://orq.ai/create-account) to get started and keep the process rolling!

**Step 1: Preparing the Environment**  
Before diving into image processing, the necessary tools must be in place. Installing the Orq SDK is quick and straightforward, setting the stage for seamless integration.

In [ ]:
!pip install orq-ai-sdk

#import
import pandas as pd
from google.colab import auth

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.8 MB/s eta 0:00:00
  Created wheel for orq-ai-sdk: filename=orq_ai_sdk-2.13.4-py3-none-any.whl size=23579 sha256=14061c73745ca2d5caab1578766599a26d0c49e368e54f6d3810842c35697661
  Stored in directory: /root/.cache/pip/wheels/24/a7/7d/823d25efb69295bec7a2355ad01cea27f2c3fb4e3adc9b2d28
Successfully built orq-ai-sdk
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


With the SDK installed, the focus shifts to setting up the client and preparing the workflow.

**Step 2: Setting Up the Orq Client**  
The Orq client bridges your environment with Orq’s powerful APIs. By authenticating with an API key, it provides access to deployments that simplify data extraction from images.

After you are logged into the platform, you can find your API key [here](https://my.orq.ai/orquesta-demos/settings/developers).

In [ ]:
import os

from orq_ai_sdk import OrqAI

client = OrqAI(
  api_key=os.environ.get("ORQ_API_KEY", "your_api_key_here"),
  environment="production"
)

In [ ]:
client.set_user(id=2024)

Once connected, the client is ready to process image files for extraction.

**Step 3: Converting Images to Base64**  
To process images with Orq’s deployments, they must first be encoded into Base64 format. This section outlines how to process a folder of .jpg and .png files, preparing them for data extraction.

To get you started, we’ve provided a Google Drive folder filled with .jpg files of receipts that you can copy and use to test and explore the workflow.

In [ ]:
import os
import base64

# Specify the folder containing image files
folder_path = '/content/drive/MyDrive/receipts_test'

# Get all .jpg and .png files from the folder
image_files = [file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png'))]

# List to store Base64-encoded data for each image
base64_images = []

# Iterate through image files and convert them to Base64
for image_file in image_files:
    file_path = os.path.join(folder_path, image_file)
    try:
        with open(file_path, 'rb') as img_file:
            # Encode the image to Base64
            base64_data = base64.b64encode(img_file.read()).decode('utf-8')
            base64_images.append(base64_data)
            print(f"Encoded {image_file} to Base64.")
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

# Output the Base64-encoded data for each image
print("Base64-encoded images ready for processing.")

Encoded 000.jpg to Base64.
Encoded 001.jpg to Base64.
Encoded 008.jpg to Base64.
Base64-encoded images ready for processing.


**Step 5: Data Extraction Deployment**

With images in Base64 format, the final step is to send each encoded image to Orq’s DataExtraction_Receipts deployment. This process extracts meaningful data fields, such as dates, vendor names, and amounts, from each image.

In [ ]:
# Iterate through each Base64-encoded image and invoke the deployment
for base64_image in base64_images:
    try:
        # Construct the invocation payload
        generation = client.deployments.invoke(
            key="DataExtraction_Receipt",
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"text": "Extract what is on the receipt", "type": "text"},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": "data:image/png;base64," + base64_image
                            },
                        },
                    ],
                }
            ],
        )

        # Print the extraction result for each image
        print(f"Extraction result: {generation.choices[0].message.content}")

    except Exception as e:
        print(f"Error invoking deployment for an image: {e}")

Extraction result: {"Date":"2018-12-25","Amount":9.0,"Category":"Office Supplies","Vendor Name":"BOOK TA .K (TAMAN DAYA) SDN BHD","Invoice Number":"TD01167104","Payment Method":"Cash"}
Extraction result: {"Date":"2018-10-19","Amount":60.3,"Category":"Home Decor","Vendor Name":"Indah Gift & Home Deco","Invoice Number":"05010035279","Payment Method":"Cash"}
Extraction result: {"Date":"2018-02-12","Amount":112.46,"Category":"Office Supplies","Vendor Name":"Perniagaan Zheng Hui","Invoice Number":"CS00082562","Payment Method":"Cash"}


**What’s Next?**  
Orq’s tools unlock powerful capabilities for handling unstructured image data. With this workflow, you can:

- Scale Data Processing: Extend the workflow to process larger datasets or integrate it into existing systems.
- Refine Model Outputs: Explore Orq's deployment configurations to optimize the data extraction process for specific image types or fields.
- Automate Further: Combine this workflow with automated pipelines to streamline tasks like financial reporting or expense management.

By bridging unstructured image data with structured outputs, Orq ensures that businesses can transform their operations and uncover new efficiencies with minimal effort.